# DSCI 619: Project 1

### Symphony Hopkins

## Introduction

For this project, we will be acting as a data scientist at NASA. We were given a data set obtained from a series of aerodynamic and acoustic tests of two and three-dimensional airfoil blade sections conducted in an anechoic wind tunnel.

This dataset has the following features:

* Frequency, in Hertzs.
* Angle of attack, in degrees.
* Chord length, in meters.
* Free-stream velocity, in meters per second.
* Suction side displacement thickness, in meters.

The only target is:
* Scaled sound pressure level, in decibels.

Our objective is to create and evaluate deep learning regression models to forecast the scaled sound pressure level.


Data Source: https://archive.ics.uci.edu/ml/datasets/Airfoil+Self-Noise

## Data Analysis

First, we will import the necessary libraries to load the dataset.

**Q1. Load the dataset, airfoil_self_noise.DAT, into memory. 
(Hint: This file is tab separated file)**

In [1]:
#importing library
import pandas as pd

#loading data
df = pd.read_table('airfoil_self_noise.dat', sep = '\t', header = None)
df.head()

,0,1,2,3,4,5
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


**Q2. Clean the data and check missing values for this dataset.**

Looking at the data table, we can see that the column names are not assigned, so we will do that here.

In [2]:
#some column names were abbreviated
df.columns = ['frequency','angle','chord_len','fsv','ssdt','sspl']
df.head()

,frequency,angle,chord_len,fsv,ssdt,sspl
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


Let's check the data types for each column to confirm they are in the correct format.

In [3]:
df.dtypes

frequency      int64
angle        float64
chord_len    float64
fsv          float64
ssdt         float64
sspl         float64
dtype: object

As we can see, the data in the columns are all numerical. Now, we will check each column to see if there are missing values.

In [4]:
df.isnull().sum(axis=0)

frequency    0
angle        0
chord_len    0
fsv          0
ssdt         0
sspl         0
dtype: int64

We can confirm that there are not any missing values. The data is now prepared to be split into training and test sets.

**Q3. Split the data into 80% of training and 20% of test dataset.**

In order to split the data, we must assign the features into variable *X*, and assign the target into variable *y*.

In [5]:
#assigning variables
X = df.drop('sspl',axis=1)
y = df['sspl']

We will now split the data into training and test datasets with 80% going into the training dataset and 20% going into the test dataset.

In [6]:
#splitting data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 2021)

For our first model, we will use simple linear regression from scikit-learn.

**Q3. Build a simple linear regression to forecast "Scaled sound pressure level" using all other features and scikit-learn package. What's the test error for this scikit-learn regression model?**

In [7]:
#importing libraries
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

#creating linear regression object
regr = linear_model.LinearRegression()

#training the model using the training dataset
regr.fit(X_train, y_train)

#making predictions using the test dataset
y_pred = regr.predict(X_test)

#computing the mean squared error to find the test error for the model
print('Mean Squared Error (MSE):', mean_squared_error(y_test, y_pred))

Mean Squared Error (MSE): 25.00175595388133


MSE helps us determine if we have a good model. An MSE of 0 indicates that our model perfectly predicts values; however, it is important to note that this rarely, if ever, happens. For our test, we received a MSE of approximately 25, so there was a difference between the predicted values and actual values. The smaller the MSE, the better the model. We will create and evaluate more models to see if we can receive a better MSE.

Source: https://en.wikipedia.org/wiki/Mean_squared_error#Interpretation

Because we have various features with different ranges, we need to normalize them so they can fall within the same range of [0,1].

**Q4. Preprocess the data using the normalization method to convert all features into the range of [0,1]**

In [8]:
#importing library
from sklearn.preprocessing import MinMaxScaler

#creating a scaler so we can transform the data to fit within the range of [0,1]
scaler = MinMaxScaler()

#normalizing the data
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Now that we have normalized the data, we can use it to create a deep learning regression model.

**Q5. Build a deep learning regression model to forecast "Scaled sound pressure level" using all other features and TensorFlow. Please use only two layers of neuron network. You choose the number of neurons to use in the first layer. What's the test error for this model?**

To create a deep learning regression model, we must first import Tensorflow.

In [9]:
#importing libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Now, we will set up the neural network. Since there are not set rules for determining the number of neurons in the first layer, we will set it to contain 6 neurons. The second layer will only contain 1 neuron because this is a regression problem and we need to aggregate all of the data from the previous layer into a single output.

In [10]:
#setting up model
model = keras.Sequential(
    [
        #first layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #second layer has 1 neuron
        layers.Dense(1)
    ]
)

model.compile(optimizer='adam',loss='mse')


Metal device set to: Apple M1


2023-02-05 17:00:53.082684: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-05 17:00:53.083341: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Using the data we normalized earlier, we will train our new model. Since we are using Tensorflow, we must convert the data into numpy arrays first. Then, we can train the data. 

In [11]:
#in order to train the model, we need to convert the data to numpy
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

#setting the random seed to reproduce the results
tf.random.set_seed(1)

#fitting the data
model.fit(x=X_train,y=y_train,batch_size=64,epochs=100,
          validation_data=(X_test,y_test))


Epoch 1/100


2023-02-05 17:00:53.190165: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-05 17:00:53.377569: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 1s 12ms/step - loss: 15667.5332 - val_loss: 15559.1377
Epoch 2/100
19/19 [==============================] - 0s 4ms/step - loss: 15651.5400 - val_loss: 15543.4219
Epoch 3/100
16/19 [========================>.....] - ETA: 0s - loss: 15667.2979

2023-02-05 17:00:53.921032: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 0s 4ms/step - loss: 15635.6846 - val_loss: 15527.6172
Epoch 4/100
19/19 [==============================] - 0s 4ms/step - loss: 15619.2676 - val_loss: 15510.7109
Epoch 5/100
19/19 [==============================] - 0s 4ms/step - loss: 15601.2275 - val_loss: 15491.3564
Epoch 6/100
19/19 [==============================] - 0s 4ms/step - loss: 15580.7402 - val_loss: 15468.9297
Epoch 7/100
19/19 [==============================] - 0s 4ms/step - loss: 15556.8701 - val_loss: 15442.4346
Epoch 8/100
19/19 [==============================] - 0s 4ms/step - loss: 15529.0010 - val_loss: 15410.8887
Epoch 9/100
19/19 [==============================] - 0s 4ms/step - loss: 15494.6416 - val_loss: 15370.5547
Epoch 10/100
19/19 [==============================] - 0s 4ms/step - loss: 15451.6855 - val_loss: 15320.9277
Epoch 11/100
19/19 [==============================] - 0s 5ms/step - loss: 15398.7598 - val_loss: 15260.7422
Epoch 12/100
19/19 [===========================

19/19 [==============================] - 0s 4ms/step - loss: 4699.5112 - val_loss: 4585.8403
Epoch 80/100
19/19 [==============================] - 0s 4ms/step - loss: 4558.9355 - val_loss: 4448.8994
Epoch 81/100
19/19 [==============================] - 0s 4ms/step - loss: 4421.6587 - val_loss: 4314.7827
Epoch 82/100
19/19 [==============================] - 0s 4ms/step - loss: 4287.3291 - val_loss: 4183.6338
Epoch 83/100
19/19 [==============================] - 0s 4ms/step - loss: 4155.7603 - val_loss: 4056.0388
Epoch 84/100
19/19 [==============================] - 0s 4ms/step - loss: 4027.8022 - val_loss: 3931.0806
Epoch 85/100
19/19 [==============================] - 0s 4ms/step - loss: 3902.4802 - val_loss: 3809.6096
Epoch 86/100
19/19 [==============================] - 0s 4ms/step - loss: 3780.5901 - val_loss: 3691.0469
Epoch 87/100
19/19 [==============================] - 0s 4ms/step - loss: 3661.8950 - val_loss: 3575.4338
Epoch 88/100
19/19 [==============================] - 0s 5m

After fitting the data, we can finally evaluate the model by calculating the Mean Squared Error (MSE).

In [12]:
#importing libraries
from sklearn.metrics import mean_squared_error,mean_absolute_error

#calculating MSE
y_pred = model.predict(X_test)
print(f'Mean Squared Error (MSE): {mean_squared_error(y_test,y_pred)}')


10/10 [==============================] - 0s 2ms/step
Mean Squared Error (MSE): 2358.551592661997


2023-02-05 17:01:02.407490: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


For this model, we received a MSE of approximately 2358, which is greater than the previous model's MSE. Let's see if we can improve the model's performance by creating two new models with adjustments: 
* The first model will have adjustments to the number of layers. 
* The second model will have adjustments to the number of neurons in each layer.

**Q6. Can you improve the model performance of Q5 by building two more models with more layers or more hidden units in each layer? Please recommend the best model from Q5 and Q6 by justifying your answer**

As we stated, the first model will have a different number of layers. Instead of two layers like our original model had, this model will contain ten layers. The first nine layers will each have 6 neurons, and the last layer will only have 1 neuron because this is a regression problem.

In [13]:
#setting up model with adjustments to the number of layers
model = keras.Sequential(
    [
        #first layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #second layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #third layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #fourth layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #fifth layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #sixth layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #seventh layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #eigth layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #ninth layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #tenth layer has 1 neuron
        layers.Dense(1)
    ]
)

model.compile(optimizer='adam',loss='mse')

#fitting the data
model.fit(x=X_train,y=y_train,batch_size=64,epochs=100,
          validation_data=(X_test,y_test))

#calculating MSE to evaluate model
y_pred = model.predict(X_test)
print(f'Mean Squared Error (MSE): {mean_squared_error(y_test,y_pred)}')

Epoch 1/100


2023-02-05 17:01:02.773860: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 1s 18ms/step - loss: 15640.6982 - val_loss: 15527.0664
Epoch 2/100
17/19 [=========================>....] - ETA: 0s - loss: 15582.5791

2023-02-05 17:01:03.297423: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 0s 8ms/step - loss: 15609.5371 - val_loss: 15488.9102
Epoch 3/100
19/19 [==============================] - 0s 8ms/step - loss: 15561.3623 - val_loss: 15426.5889
Epoch 4/100
19/19 [==============================] - 0s 8ms/step - loss: 15477.8096 - val_loss: 15312.9746
Epoch 5/100
19/19 [==============================] - 0s 8ms/step - loss: 15314.6865 - val_loss: 15077.3467
Epoch 6/100
19/19 [==============================] - 0s 8ms/step - loss: 14955.3311 - val_loss: 14538.4170
Epoch 7/100
19/19 [==============================] - 0s 8ms/step - loss: 14128.5449 - val_loss: 13304.5752
Epoch 8/100
19/19 [==============================] - 0s 8ms/step - loss: 12272.7324 - val_loss: 10633.4395
Epoch 9/100
19/19 [==============================] - 0s 8ms/step - loss: 8552.4346 - val_loss: 5787.1929
Epoch 10/100
19/19 [==============================] - 0s 8ms/step - loss: 3223.5815 - val_loss: 973.3898
Epoch 11/100
19/19 [==============================] -

19/19 [==============================] - 0s 8ms/step - loss: 35.0313 - val_loss: 37.3370
Epoch 82/100
19/19 [==============================] - 0s 8ms/step - loss: 34.3681 - val_loss: 36.4218
Epoch 83/100
19/19 [==============================] - 0s 8ms/step - loss: 33.3228 - val_loss: 35.4298
Epoch 84/100
19/19 [==============================] - 0s 8ms/step - loss: 32.6744 - val_loss: 34.9206
Epoch 85/100
19/19 [==============================] - 0s 8ms/step - loss: 31.9301 - val_loss: 34.7938
Epoch 86/100
19/19 [==============================] - 0s 8ms/step - loss: 31.5862 - val_loss: 33.5385
Epoch 87/100
19/19 [==============================] - 0s 8ms/step - loss: 30.7541 - val_loss: 33.1440
Epoch 88/100
19/19 [==============================] - 0s 8ms/step - loss: 30.2415 - val_loss: 32.5805
Epoch 89/100
19/19 [==============================] - 0s 8ms/step - loss: 29.8209 - val_loss: 32.0556
Epoch 90/100
19/19 [==============================] - 0s 8ms/step - loss: 29.2714 - val_loss: 3

2023-02-05 17:01:18.683989: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


After adjusting the number of layers, we can see that the MSE decreased from 2358 to 29. This suggests the model's performance improved. Let's see if we can improve the model's performance by adjusting the number of neurons. For our second model, we will have 60 neurons in the first layer and only 1 neuron in the last layer.

In [14]:
#setting up model with adjustments to the number of neurons in the first layer
model = keras.Sequential(
    [
        #first layer has 60 neurons
        layers.Dense(60, activation='relu'),
        #second layer has 1 neuron
        layers.Dense(1)
    ]
)

model.compile(optimizer='adam',loss='mse')

#fitting the data
model.fit(x=X_train,y=y_train,batch_size=64,epochs=100,
          validation_data=(X_test,y_test))

#calculating MSE to evaluate model
y_pred = model.predict(X_test)
print(f'Mean Squared Error (MSE): {mean_squared_error(y_test,y_pred)}')


Epoch 1/100
15/19 [======================>.......] - ETA: 0s - loss: 15604.1494

2023-02-05 17:01:18.897124: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 0s 8ms/step - loss: 15600.1211 - val_loss: 15462.5195
Epoch 2/100
19/19 [==============================] - 0s 5ms/step - loss: 15522.7031 - val_loss: 15378.3438
Epoch 3/100
16/19 [========================>.....] - ETA: 0s - loss: 15466.4355

2023-02-05 17:01:19.126368: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 0s 4ms/step - loss: 15428.0898 - val_loss: 15271.5674
Epoch 4/100
19/19 [==============================] - 0s 5ms/step - loss: 15307.7520 - val_loss: 15136.8945
Epoch 5/100
19/19 [==============================] - 0s 5ms/step - loss: 15158.4492 - val_loss: 14968.8135
Epoch 6/100
19/19 [==============================] - 0s 5ms/step - loss: 14972.9209 - val_loss: 14762.2451
Epoch 7/100
19/19 [==============================] - 0s 5ms/step - loss: 14747.9844 - val_loss: 14519.0283
Epoch 8/100
19/19 [==============================] - 0s 5ms/step - loss: 14488.9268 - val_loss: 14245.2441
Epoch 9/100
19/19 [==============================] - 0s 5ms/step - loss: 14199.9316 - val_loss: 13941.5000
Epoch 10/100
19/19 [==============================] - 0s 5ms/step - loss: 13881.6133 - val_loss: 13608.2559
Epoch 11/100
19/19 [==============================] - 0s 5ms/step - loss: 13534.0908 - val_loss: 13246.9248
Epoch 12/100
19/19 [===========================

19/19 [==============================] - 0s 4ms/step - loss: 792.7700 - val_loss: 832.9720
Epoch 81/100
19/19 [==============================] - 0s 4ms/step - loss: 786.5666 - val_loss: 826.3944
Epoch 82/100
19/19 [==============================] - 0s 4ms/step - loss: 780.2109 - val_loss: 819.8427
Epoch 83/100
19/19 [==============================] - 0s 4ms/step - loss: 774.1122 - val_loss: 813.0063
Epoch 84/100
19/19 [==============================] - 0s 4ms/step - loss: 767.6760 - val_loss: 806.3754
Epoch 85/100
19/19 [==============================] - 0s 4ms/step - loss: 761.4174 - val_loss: 799.5978
Epoch 86/100
19/19 [==============================] - 0s 4ms/step - loss: 755.1213 - val_loss: 792.7857
Epoch 87/100
19/19 [==============================] - 0s 4ms/step - loss: 748.6183 - val_loss: 786.2313
Epoch 88/100
19/19 [==============================] - 0s 4ms/step - loss: 742.5316 - val_loss: 779.2928
Epoch 89/100
19/19 [==============================] - 0s 4ms/step - loss: 736

2023-02-05 17:01:27.778668: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


As we can see, the MSE changed to 697, which is not an improvement from the previous model (MSE = 29). From the models we created from Q5 and Q6, we recommend the model with more layers as the best model. Let's see if we can further improve that model by using a larger number of epochs. For the new model, we will increase the number of epochs from 100 to 200.

**Q7. Can you improve the best model in Q6 by training the model for longer using a larger epochs?**

In [15]:
#setting up model with adjustments to the number of epochs
model = keras.Sequential(
    [
        #first layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #second layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #third layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #fourth layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #fifth layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #sixth layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #seventh layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #eigth layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #ninth layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #tenth layer has 1 neuron
        layers.Dense(1)
    ]
)

model.compile(optimizer='adam',loss='mse')

#fitting the data
model.fit(x=X_train,y=y_train,batch_size=64,epochs=200,
          validation_data=(X_test,y_test))

#calculating MSE to evaluate model
y_pred = model.predict(X_test)
print(f'Mean Squared Error (MSE): {mean_squared_error(y_test,y_pred)}')

Epoch 1/200
 8/19 [===========>..................] - ETA: 0s - loss: 15630.1816

2023-02-05 17:01:28.159245: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 1s 16ms/step - loss: 15641.7920 - val_loss: 15530.2617
Epoch 2/200
17/19 [=========================>....] - ETA: 0s - loss: 15588.5459

2023-02-05 17:01:28.493923: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 0s 8ms/step - loss: 15615.8877 - val_loss: 15499.8730
Epoch 3/200
19/19 [==============================] - 0s 8ms/step - loss: 15576.9375 - val_loss: 15448.9727
Epoch 4/200
19/19 [==============================] - 0s 8ms/step - loss: 15504.2207 - val_loss: 15344.6074
Epoch 5/200
19/19 [==============================] - 0s 8ms/step - loss: 15341.2207 - val_loss: 15093.0791
Epoch 6/200
19/19 [==============================] - 0s 8ms/step - loss: 14925.3389 - val_loss: 14422.6904
Epoch 7/200
19/19 [==============================] - 0s 8ms/step - loss: 13805.9961 - val_loss: 12648.6846
Epoch 8/200
19/19 [==============================] - 0s 8ms/step - loss: 11008.6611 - val_loss: 8533.0684
Epoch 9/200
19/19 [==============================] - 0s 8ms/step - loss: 5574.8340 - val_loss: 2328.8940
Epoch 10/200
19/19 [==============================] - 0s 8ms/step - loss: 1169.9672 - val_loss: 1024.2793
Epoch 11/200
19/19 [==============================] -

19/19 [==============================] - 0s 9ms/step - loss: 23.0702 - val_loss: 26.4031
Epoch 82/200
19/19 [==============================] - 0s 8ms/step - loss: 22.9176 - val_loss: 26.4988
Epoch 83/200
19/19 [==============================] - 0s 7ms/step - loss: 22.8512 - val_loss: 26.0155
Epoch 84/200
19/19 [==============================] - 0s 8ms/step - loss: 22.5996 - val_loss: 25.8392
Epoch 85/200
19/19 [==============================] - 0s 8ms/step - loss: 22.3846 - val_loss: 26.0859
Epoch 86/200
19/19 [==============================] - 0s 9ms/step - loss: 22.8870 - val_loss: 25.4930
Epoch 87/200
19/19 [==============================] - 0s 8ms/step - loss: 22.3503 - val_loss: 25.4050
Epoch 88/200
19/19 [==============================] - 0s 8ms/step - loss: 22.1293 - val_loss: 25.2401
Epoch 89/200
19/19 [==============================] - 0s 8ms/step - loss: 22.1096 - val_loss: 25.1474
Epoch 90/200
19/19 [==============================] - 0s 8ms/step - loss: 21.9168 - val_loss: 2

Epoch 161/200
19/19 [==============================] - 0s 8ms/step - loss: 19.0173 - val_loss: 22.0107
Epoch 162/200
19/19 [==============================] - 0s 8ms/step - loss: 18.8728 - val_loss: 21.9938
Epoch 163/200
19/19 [==============================] - 0s 8ms/step - loss: 18.9472 - val_loss: 23.0511
Epoch 164/200
19/19 [==============================] - 0s 8ms/step - loss: 19.5170 - val_loss: 22.2659
Epoch 165/200
19/19 [==============================] - 0s 8ms/step - loss: 19.0306 - val_loss: 21.4513
Epoch 166/200
19/19 [==============================] - 0s 8ms/step - loss: 18.7404 - val_loss: 21.3964
Epoch 167/200
19/19 [==============================] - 0s 9ms/step - loss: 18.8093 - val_loss: 21.4957
Epoch 168/200
19/19 [==============================] - 0s 8ms/step - loss: 18.8555 - val_loss: 21.5502
Epoch 169/200
19/19 [==============================] - 0s 8ms/step - loss: 18.8274 - val_loss: 21.3336
Epoch 170/200
19/19 [==============================] - 0s 8ms/step - loss

2023-02-05 17:01:58.726011: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


After increasing the number of epochs to 200, we received a MSE of 21, which means we were successful in improving the best model from Q6. Let's see if using a different optimizer will give us better results.

**Q8 Can you improve the best model in Q6 by using different optimizer?**

So far, we have used Adam for algorithm optimization; but for this model, we will use SGD. Both optimizers have different strengths and weaknesses. There are many articles that discuss this: [Deep Learning Optimizers: SGD with momentum, Adagrad, Adadelta, Adam optimizer](https://towardsdatascience.com/deep-learning-optimizers-436171c9e23f),
[A 2021 Guide to improving CNNs-Optimizers: Adam vs SGD](https://medium.com/geekculture/a-2021-guide-to-improving-cnns-optimizers-adam-vs-sgd-495848ac6008), [An overview of gradient descent optimization algorithms](https://www.ruder.io/optimizing-gradient-descent/#adam). We will not go into full detail, but we will see if changing optimizers can improve the best model from Q6.

In [16]:
#setting up model with an adjustment to the type of optimizer
model = keras.Sequential(
    [
        #first layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #second layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #third layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #fourth layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #fifth layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #sixth layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #seventh layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #eigth layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #ninth layer has 6 neurons
        layers.Dense(6, activation='relu'),
        #tenth layer has 1 neuron
        layers.Dense(1)
    ]
)

model.compile(optimizer='SGD',loss='mse')

#fitting the data
model.fit(x=X_train,y=y_train,batch_size=64,epochs=100,
          validation_data=(X_test,y_test))

#calculating MSE to evaluate model
y_pred = model.predict(X_test)
print(f'Mean Squared Error (MSE): {mean_squared_error(y_test,y_pred)}')

Epoch 1/100
 1/19 [>.............................] - ETA: 6s - loss: 15961.4453

2023-02-05 17:01:59.015247: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 1s 18ms/step - loss: 40704897024.0000 - val_loss: 159104640.0000
Epoch 2/100
19/19 [==============================] - ETA: 0s - loss: 113759424.0000

2023-02-05 17:01:59.438212: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 0s 7ms/step - loss: 113759424.0000 - val_loss: 73834640.0000
Epoch 3/100
19/19 [==============================] - 0s 11ms/step - loss: 52793120.0000 - val_loss: 34263612.0000
Epoch 4/100
19/19 [==============================] - 0s 10ms/step - loss: 24500198.0000 - val_loss: 15899915.0000
Epoch 5/100
19/19 [==============================] - 0s 7ms/step - loss: 11369971.0000 - val_loss: 7378094.0000
Epoch 6/100
19/19 [==============================] - 0s 7ms/step - loss: 5276555.0000 - val_loss: 3423574.7500
Epoch 7/100
19/19 [==============================] - 0s 7ms/step - loss: 2448769.7500 - val_loss: 1588507.1250
Epoch 8/100
19/19 [==============================] - 0s 7ms/step - loss: 1136448.2500 - val_loss: 737004.0000
Epoch 9/100
19/19 [==============================] - 0s 7ms/step - loss: 527430.8750 - val_loss: 341897.4375
Epoch 10/100
19/19 [==============================] - 0s 7ms/step - loss: 244793.0156 - val_loss: 158596.7344
Epoch 1

19/19 [==============================] - 0s 7ms/step - loss: 47.2863 - val_loss: 48.7620
Epoch 82/100
19/19 [==============================] - 0s 9ms/step - loss: 47.2791 - val_loss: 48.7586
Epoch 83/100
19/19 [==============================] - 0s 8ms/step - loss: 47.2814 - val_loss: 48.7685
Epoch 84/100
19/19 [==============================] - 0s 7ms/step - loss: 47.2760 - val_loss: 48.7649
Epoch 85/100
19/19 [==============================] - 0s 7ms/step - loss: 47.2793 - val_loss: 48.7769
Epoch 86/100
19/19 [==============================] - 0s 7ms/step - loss: 47.2794 - val_loss: 48.7704
Epoch 87/100
19/19 [==============================] - 0s 7ms/step - loss: 47.2786 - val_loss: 48.7741
Epoch 88/100
19/19 [==============================] - 0s 7ms/step - loss: 47.2761 - val_loss: 48.7678
Epoch 89/100
19/19 [==============================] - 0s 7ms/step - loss: 47.2789 - val_loss: 48.7673
Epoch 90/100
19/19 [==============================] - 0s 7ms/step - loss: 47.2721 - val_loss: 4

2023-02-05 17:02:13.684736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


After changing the optimizer, the MSE changed to 49. When we compare this adjusted model to the best model from Q6, we can see that changing optimizers did not improve the model's performance.

## Summary

From the models we created, we have the following MSE (ranked in increasing order):
* Fourth Tensor Model (Q7 - Increased Layers & Epochs): 21
* Scikit-learn Model (Q3): 25
* Second Tensor Model (Q6 - Increased Layers): 29
* Fifth Tensor Model (Q8 - Increased Layers & Changed Optimizer): 49
* Third Tensor Model (Q6 - Increased Neurons): 697
* First Tensor Model (Q5): 2358

Out of all of the models we created, the model's performance was the best when we increased the number of layers and neurons because it had the lowest MSE. While MSE is a way we can evaluate models, we would like to investigate more questions in the future (e.g., Are any of these models under-fitting/over-fitting the data?) before we recommend a model to NASA.